In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import sys
sys.path.append('/Users/mregnier/Desktop/PhD Regnier/mypackages')


import Acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils
import myPCG as mypcg

from qubic import SpectroImLib as sp
from importlib import reload
import gc
import copy
from pyoperators import *


# PyOperators packages
from pyoperators import (
    BlockColumnOperator, BlockDiagonalOperator, BlockRowOperator,
    CompositionOperator, DiagonalOperator, I, IdentityOperator,
    MPIDistributionIdentityOperator, MPI, proxy_group, ReshapeOperator,
    rule_manager, pcg, Operator)

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")

# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('font', size=15)
rc('figure', figsize=(20, 8))


In [ ]:
def get_H(Nf_TOD, nside=256, nside_init=1024):
    print('#################### {} ###############'.format(Nf_TOD))
    Nf_recon = 2
    fact_sub = 5


    # Repository for dictionary
    global_dir = Qubic_DataDir()
    print(global_dir)
    dictfilename = global_dir + 'dicts/pipeline_demo.dict'

    # Read dictionary chosen
    d_TOD = qubic.qubicdict.qubicDict()
    d_TOD.read_from_file(dictfilename)

    d_TOD['nf_recon'] = Nf_TOD
    d_TOD['nf_sub'] = Nf_TOD
    d_TOD['nside'] = nside_init
    npix=12*d_TOD['nside']**2
    d_TOD['RA_center'] = 0
    d_TOD['DEC_center'] = -57
    center = qubic.equ2gal(d_TOD['RA_center'], d_TOD['DEC_center'])
    d_TOD['effective_duration'] = effective_duration
    d_TOD['npointings'] = npointings
    d_TOD['tol'] = 5e-4
    d_TOD['filter_nu'] = band * 1e9
    d_TOD['photon_noise'] = False#not noiseless
    d_TOD['noiseless'] = noiseless
    d_TOD['config'] = 'FI'
    d_TOD['MultiBand'] = True
    d_TOD['planck'] = True
    d_TOD['dtheta'] = 15
    #d_TOD['synthbeam_dtype'] = float

    print('*************** Noise **************')
    print('Noisless:      {}'.format(d_TOD['noiseless']))
    print('Photon Noise:  {}'.format(d_TOD['photon_noise']))
    print('************************************')

    sky_config = {'cmb':42}
    #sky_config = {'dust':'d0'}
    #sky_config = {'cmb':3, 'dust':'d0'}

    reload(Acq)
    qubic_acquisition = Acq.QubicIntegrated(d_TOD, Nsub=Nf_TOD, Nrec=Nf_TOD)
    _, nus_edge, nus, deltas, Delta, _ = qubic.compute_freq(int(d_TOD['filter_nu']/1e9), Nfreq=qubic_acquisition.Nsub)
    cov = qubic_acquisition.get_coverage()
    covnorm = cov/cov.max()
    threshold = 0
    #seenpix = generate_circular_mask(256, 10, center[0], center[1])
    seenpix = covnorm > threshold
    notseenpix = covnorm < threshold

    # Create Nsub maps
    m_sub_in = qubic_acquisition.get_PySM_maps(sky_config, qubic_acquisition.allnus)
    print(m_sub_in.shape)
    
    ### We need to degrade High Nside maps used for TOD to the "normal" nside that will be used for Planck
    m_sub = np.zeros((Nf_TOD, 12*nside**2, 3))
    for k in range(3):
        for i in range(Nf_TOD):
            m_sub[i,:,k] = hp.ud_grade(m_sub_in[i,:,k], nside)


    if Nf_TOD == 1:
        m_sub = np.reshape(m_sub, (12*nside**2, 3))
        
    d_TOD['nside'] = nside
    qubic_acquisition = Acq.QubicIntegrated(d_TOD, Nsub=Nf_TOD, Nrec=Nf_TOD)
    ptg = qubic_acquisition.sampling
    
    # Create operator
    H = qubic_acquisition.get_operator(convolution=True, myfwhm=None, fixed_data=None)
    del qubic_acquisition
    gc.collect()
    return H, m_sub, nus_edge, nus, deltas, Delta, ptg


In [ ]:
# reload(Acq)
# nside = 1024
# relative_bandwidth = 0.25
# band = 220
# band_planck = 217
# seed = 42
# noiseless = True
# effective_duration = 3e0
# rwplanck = 1.
# tol = 1e-40
# noise_planck_level = 0
# noise_qubic_level = 0

# npointings = 3000
# test_nftod = np.append([1,2,3,4], np.arange(1, 10)*5)
# alltod = np.zeros((len(test_nftod), 992*npointings))

# print(test_nftod)


# for i in range(len(test_nftod)):
#     H_TOD, m_sub = get_H(test_nftod[i], nside=nside)
#     TOD_QUBIC = H_TOD(m_sub)
#     alltod[i,:] = TOD_QUBIC.ravel()

#     del H_TOD
#     gc.collect()


In [ ]:
# figure(figsize=(15, 8))

# subplot(1,2,1)
# errorbar(test_nftod, np.mean(alltod, axis=1), yerr=np.std(alltod, axis=1)/np.sqrt(npointings*992), fmt='ro')
# xlabel('Nf_TOD')
# ylabel('Mean TOD (err on mean)')
# yscale('log')
# title(nside)
# #ylim(1e-19, 1e-17)

# subplot(1,2,2)
# diff = np.zeros(len(test_nftod))
# for i in range(len(test_nftod)):
#     diff[i] = np.std(alltod[i,:] - alltod[-1,:])
# plot(test_nftod, diff/diff[0], 'bo-')
# yscale('log')
# xlabel('Nf_TOD')
# ylabel('RMS(difference to Nf={}) normed to Nf={}'.format(test_nftod[-1], test_nftod[0]))
# title(nside)
# ylim(1e-2, 1)

# Let's look at the difference in TOD among the sub-bands

In [ ]:
reload(Acq)
relative_bandwidth = 0.25
band = 220
band_planck = 217
seed = 42
noiseless = True
effective_duration = 3e0
rwplanck = 1.
tol = 1e-40
noise_planck_level = 0
noise_qubic_level = 0
npointings = 3000

nside = 256#1024
nside_init = 256#1024
nf_tod = 15

H, m, nus_edge, nus, deltas, Delta, ptg = get_H(nf_tod, nside=nside, nside_init=nside_init)
m = m*0+1
tod = H(m)

sh = np.shape(tod)
alltod = np.zeros((nf_tod, sh[0], sh[1]))

for i in range(nf_tod):
    alltod[i,:,:] = H.operands[i](m[i,:,:])

del H
gc.collect()

In [ ]:
rc('font', size=15)
rc('figure', figsize=(20, 8))
figure(figsize=(20, 8))

rmsdiff = np.zeros((nf_tod, sh[0]))
for i in range(nf_tod):
    diff = tod - alltod[i,:,:]/deltas[i]*Delta
    rmsdiff[i,:] = np.std(diff, axis=1) / np.std(tod, axis=1)

subplot(1,2,1)
for i in range(nf_tod):
    plot(rmsdiff[i,:], label='band {}'.format(i))
legend()
xlabel('TES#')
ylabel('RMS (TOD-TOD_i) / RMS(TOD)')
title('Nsub = {} ; Nside = {} ; Nside_init={}'.format(nf_tod, nside, nside_init))
axhline(y=1, color='k', ls='--')
ylim(0,1.5)

subplot(1,2,2)
errorbar(np.arange(nf_tod), np.mean(rmsdiff, axis=1), yerr=np.std(rmsdiff, axis=1), fmt='ro')
xlabel('Sub-Band number')
ylabel('Relative RMS')
title('Nsub = {} ; Nside = {} ; Nside_init={}'.format(nf_tod, nside, nside_init))
axhline(y=1, color='k', ls='--')
ylim(0,1.5)

tight_layout()

In [ ]:
rc('font', size=15)
rc('figure', figsize=(20, 8))
## Let's investigate a single TES
from qubic import fibtools as ft
mytes = 0
plot(tod[mytes,:])


figure()
ax = subplot(1,2,1)
for i in range(nf_tod):
    p=plot(tod[mytes,:], alltod[i, mytes,:], '.', alpha=1, label='band {}'.format(i))
    #p=plot(tod[mytes,:], alltod[i, mytes,:], '.', alpha=1, label='band {}'.format(i))
#     x,y,dx,dy,_=ft.profile(tod[mytes,:]/Delta, alltod[i, mytes,:]/deltas[i])
#     errorbar(x,y,xerr=dx, yerr=dy, fmt='o-', color=p[0].get_color())

mini = -np.std(tod)
maxi = 2*np.std(tod)
xlim(mini, maxi)
ylim(mini, maxi)
plot([mini, maxi], [mini, maxi], 'k--')
ax.set_aspect('equal')
xlabel('TOD')
ylabel('tod_i')
plot()
legend(fontsize=8)

ax = subplot(1,2,2)
for i in range(nf_tod):
    p=plot(tod[mytes,:]/Delta, alltod[i, mytes,:]/deltas[i], '.', alpha=1, label='band {}'.format(i))
    #p=plot(tod[mytes,:], alltod[i, mytes,:], '.', alpha=1, label='band {}'.format(i))
#     x,y,dx,dy,_=ft.profile(tod[mytes,:]/Delta, alltod[i, mytes,:]/deltas[i])
#     errorbar(x,y,xerr=dx, yerr=dy, fmt='o-', color=p[0].get_color())

mini = -np.std(tod/Delta)
maxi = 2*np.std(tod/Delta)
xlim(mini, maxi)
ylim(mini, maxi)
plot([mini, maxi], [mini, maxi], 'k--')
ax.set_aspect('equal')
xlabel('TOD/Delta')
ylabel('tod_i / delta_i')
plot()
legend(fontsize=8)



In [ ]:
ax = subplot(1,3,1)
al=0.4
plot(alltod[0,mytes,:],alltod[1,mytes,:], 'o', alpha=al, label='1 vs 0')
plot(alltod[0,mytes,:],alltod[7,mytes,:], 'o', alpha=al, label='7 vs 0')
plot(alltod[0,mytes,:],alltod[14,mytes,:], 'o', alpha=al, label='14 vs 0')
mini = -np.std(alltod[0,mytes,:])
maxi = 5*np.std(alltod[0,mytes,:])
xlim(mini, maxi)
ylim(mini, maxi)
plot([mini, maxi], [mini, maxi], 'k--')
ax.set_aspect('equal')
legend()
xlabel('tod_0')
ylabel('tod_i')

ax = subplot(1,3,2)
al=0.4
plot(alltod[0,mytes,:]/deltas[0],alltod[1,mytes,:]/deltas[1], 'o', alpha=al, label='1 vs 0')
plot(alltod[0,mytes,:]/deltas[0],alltod[7,mytes,:]/deltas[7], 'o', alpha=al, label='7 vs 0')
plot(alltod[0,mytes,:]/deltas[0],alltod[14,mytes,:]/deltas[14], 'o', alpha=al, label='14 vs 0')
mini = -np.std(alltod[0,mytes,:]/deltas[0])
maxi = 5*np.std(alltod[0,mytes,:]/deltas[0])
xlim(mini, maxi)
ylim(mini, maxi)
plot([mini, maxi], [mini, maxi], 'k--')
ax.set_aspect('equal')
legend()
xlabel('tod_0 / delta_0')
ylabel('tod_i / delta_i')

ax = subplot(1,3,3)
al=0.4
plot(alltod[0,mytes,:]*deltas[0],alltod[1,mytes,:]*deltas[1], 'o', alpha=al, label='1 vs 0')
plot(alltod[0,mytes,:]*deltas[0],alltod[7,mytes,:]*deltas[7], 'o', alpha=al, label='7 vs 0')
plot(alltod[0,mytes,:]*deltas[0],alltod[14,mytes,:]*deltas[14], 'o', alpha=al, label='14 vs 0')
mini = -np.std(alltod[0,mytes,:]*deltas[0])
maxi = 5*np.std(alltod[0,mytes,:]*deltas[0])
xlim(mini, maxi)
ylim(mini, maxi)
plot([mini, maxi], [mini, maxi], 'k--')
ax.set_aspect('equal')
legend()
xlabel('tod_0 * delta_0')
ylabel('tod_i * delta_i')


tight_layout()

In [ ]:
import sys
rep = '/Users/hamilton/Python/GitQubic/qubic/qubic/scripts/DiversJC/CalibSalta/'
if rep not in sys.path:
    sys.path.append(rep)
import fitting

ax = subplot(1,2,1)
al=0.4
mini = -3*np.std(alltod[0,mytes,:])
maxi = 5*np.std(alltod[0,mytes,:])
fitting.myhist(alltod[0,mytes,:]-alltod[1,mytes,:], bins=100, range=[mini, maxi], alpha=al, label='1 vs 0: ')
fitting.myhist(alltod[0,mytes,:]-alltod[7,mytes,:], bins=100, range=[mini, maxi], alpha=al, label='7 vs 0: ')
fitting.myhist(alltod[0,mytes,:]-alltod[14,mytes,:], bins=100, range=[mini, maxi], alpha=al, label='14 vs 0: ')
xlim(mini, maxi)
legend()
xlabel('tod_0 - tod_i')


ax = subplot(1,2,2)
al=0.4
mini = -3*np.std(alltod[0,mytes,:]/deltas[0])
maxi = 5*np.std(alltod[0,mytes,:]/deltas[0])
fitting.myhist(alltod[0,mytes,:]/deltas[0]-alltod[1,mytes,:]/deltas[1], bins=100, range=[mini, maxi], alpha=al, label='1 vs 0: ')
fitting.myhist(alltod[0,mytes,:]/deltas[0]-alltod[7,mytes,:]/deltas[7], bins=100, range=[mini, maxi], alpha=al, label='7 vs 0: ')
fitting.myhist(alltod[0,mytes,:]/deltas[0]-alltod[14,mytes,:]/deltas[14], bins=100, range=[mini, maxi], alpha=al, label='14 vs 0: ')
xlim(mini, maxi)
legend()
xlabel('(tod_0 / delta_0) - (tod_i / delta_i)')

tight_layout()

In [ ]:

mydeltas = deltas.copy()



ax = subplot(1,2,2)
al=0.4
mini = -3*np.std(alltod[0,mytes,:]/mydeltas[0])
maxi = 5*np.std(alltod[0,mytes,:]/mydeltas[0])
fitting.myhist(alltod[0,mytes,:]/mydeltas[0]-alltod[1,mytes,:]/mydeltas[1], bins=100, range=[mini, maxi], alpha=al, label='1 vs 0: ')
fitting.myhist(alltod[0,mytes,:]/mydeltas[0]-alltod[7,mytes,:]/mydeltas[7], bins=100, range=[mini, maxi], alpha=al, label='7 vs 0: ')
fitting.myhist(alltod[0,mytes,:]/mydeltas[0]-alltod[14,mytes,:]/mydeltas[14], bins=100, range=[mini, maxi], alpha=al, label='14 vs 0: ')
xlim(mini, maxi)
legend()
xlabel('(tod_0 / delta_0) - (tod_i / delta_i)')

tight_layout()

In [ ]:
mm.shape

In [ ]:


H, mm, nus_edge, nus, deltas, Delta, ptg = get_H(2, nside=256, nside_init=256)
mm = mm*0+1
#mm[:,:,0] = 1
mytod = H(mm)
hp.mollview(mm[0,:,0], sub=(2,2,1), title='I',min=0,max=2)
hp.mollview(mm[0,:,1], sub=(2,2,2), title='Q',min=0,max=2)
hp.mollview(mm[0,:,2], sub=(2,2,3), title='U',min=0,max=2)
subplot(2,2,4)
plot(mytod[0,:])
suptitle('Wide Band operator', fontsize=20, y=1.05)
tight_layout()

In [ ]:
H0= H.operands[0]
C0 = H0.operands[-1]
newm = C0(mm[0,:,:])
hp.mollview(newm[:,0], sub=(2,2,1),min=0,max=2)
hp.mollview(newm[:,1], sub=(2,2,2),min=0,max=2)
hp.mollview(newm[:,2], sub=(2,2,3),min=0,max=2)
tod0 = H0(mm[0,:,:])
subplot(2,2,4)
plot(tod0[0,:])
suptitle('sub-band 0', fontsize=20, y=1.05)

tight_layout()


figure()
hp.mollview(np.log10(np.abs(newm[:,0]-mm[0,:,0])), sub=(1,3,1))
hp.mollview(np.log10(np.abs(newm[:,0]-mm[0,:,0])), sub=(1,3,2))
hp.mollview(np.log10(np.abs(newm[:,0]-mm[0,:,0])), sub=(1,3,3))

In [ ]:
H0noconv = CompositionOperator(H0.operands[:-1])
hp.mollview(mm[0,:,0], sub=(2,2,1),min=0,max=2)
hp.mollview(mm[0,:,1], sub=(2,2,2),min=0,max=2)
hp.mollview(mm[0,:,2], sub=(2,2,3),min=0,max=2)
tod0 = H0noconv(mm[0,:,:])
subplot(2,2,4)
plot(tod0[0,:])
suptitle('sub-band 0 - No Conv - applied to m=1', fontsize=20, y=1.05)

tight_layout()

In [ ]:
proj_op = H0noconv.operands[-1]
bla = proj_op(mm[0,:,:])
print(bla.shape)
subplot(1,4,1)
title('I')
plot(bla[0,:,0])
subplot(1,4,2)
title('Q')
plot(bla[0,:,1])
subplot(1,4,3)
title('U')
plot(bla[0,:,2])
suptitle('sub-band 0 - No Conv - applied to m=1 - '+str(proj_op), fontsize=20, y=1.0)

### We use the repeat pointing strategy so the HWP has three positions for 1000 pointings each
subplot(1,4,4)
plot(ptg.angle_hwp)
ylabel('HWP Angle')
tight_layout()

figure()
# Now let's ee this in more details
angles_hwp = np.unique(ptg.angle_hwp)

subplot(2,2,1)
title('Q')
for a in angles_hwp:
    ok = ptg.angle_hwp == a
    plot(bla[0,ok,1]-a*100, label='Angle HWP = {0:3.1f} (shifted for visualization)'.format(a))
legend()

subplot(2,2,2)
title('U')
for a in angles_hwp:
    ok = ptg.angle_hwp == a
    plot(bla[0,ok,2]-a*100, label='Angle HWP = {0:3.1f} (shifted for visualization)'.format(a))
legend()

subplot(2,2,3)
title('Q')
for i in range(len(angles_hwp)):
    ok0 = ptg.angle_hwp == angles_hwp[i]
    ok1 = ptg.angle_hwp == angles_hwp[(i+1) % len(angles_hwp)]
    plot(bla[0,ok1,1]-bla[0,ok0,1]+i, label='Angle HWP = {0:3.1f} - {1:3.1f} (shifted for visualization)'.format(angles_hwp[i], angles_hwp[(i+1) % len(angles_hwp)]))
legend()

subplot(2,2,4)
title('U')
for i in range(len(angles_hwp)):
    ok0 = ptg.angle_hwp == angles_hwp[i]
    ok1 = ptg.angle_hwp == angles_hwp[(i+1) % len(angles_hwp)]
    plot(bla[0,ok1,2]-bla[0,ok0,2]+i, label='Angle HWP = {0:3.1f} - {1:3.1f} (shifted for visualization)'.format(angles_hwp[i], angles_hwp[(i+1) % len(angles_hwp)]))
legend()



In [ ]:
subplot(2,2,2)
for i,a in enumerate(angles_hwp):
    ok = ptg.angle_hwp == a
    plot(bla[0,ok,1], bla[0,ok,2], '.', label='Angle HWP = {0:3.1f} (shifted for visualization)'.format(a), ms=25-i*8)
legend()
xlabel('Q')
xlabel('U')

In [ ]:
H0

In [ ]:
print(H0.operands[3].data.shape)
subplot(1,2,1)
plot(H0.operands[3].data[0,:,0,0],label='I', lw=4)
plot(H0.operands[3].data[0,:,0,1],label='Q')
plot(H0.operands[3].data[0,:,0,2],label='U')
legend()

subplot(1,2,2)
plot(np.cos(np.radians(ptg.angle_hwp)), label='Cos(HWP)')
plot(np.sin(np.radians(ptg.angle_hwp)), label='Sin(HWP)')
legend()

The non-constant behaviour of the TOD when all I,Q,U are zero actually comes from the fact that Q and U maps are in a "weird" reference frame. Each pixel is actually in a distinct reference frame for nice visualization. But as a result, when one wants to get the actual Q and U seen by a detectors, one needs to apply rotations for each pixel that turns them into the observer's reference frame. So our map=1 for Q and U on the screen does not correspond to 1 for the observer and this is (likely to be) the source of this behaviour. So I dont think this is a good path to follow. The problem is elsewhere finally :-(

### Let's try something else
We take pure temperature (to avoid the effects above) and make many sub-frequencies for integration and we only look at a single detector.

In [ ]:
nsubtod = 20
H, mm, nus_edge, nus, deltas, Delta, ptg = get_H(nsubtod, nside=256, nside_init=1024)
mm[:,:,1:] = 0
hp.mollview(mm[0,:,0], sub=(2,3,1), title='I')
hp.mollview(mm[0,:,1], sub=(2,3,2), title='Q',min=0,max=2)
hp.mollview(mm[0,:,2], sub=(2,3,3), title='U',min=0,max=2)

mytod = np.zeros((nsubtod, 3000))
for i,h in enumerate(H.operands):
    mytod[i,:] = h(mm[i,:,:])[0,:]

subplot(2,2,4)
plot(mytod[0,:])
plot(mytod[1,:])
suptitle('Wide Band operator', fontsize=20, y=1.05)
tight_layout()

del h
gc.collect()

In [ ]:
def myplt(x,y,dx,fmt='o',**kwargs):
    p=errorbar(x, y, xerr=dx, fmt=fmt,**kwargs)
    plot(x, y, drawstyle='steps-mid', color=p[0].get_color())
    return p

subplot(1,2,1)
for k in range(4):
    p=myplt(nus, mytod[:,k],deltas/2, label='Det{} nsinit=1024, ns=256'.format(k))
    myplt(nus, mytod1024[:,k],deltas/2, color=p[0].get_color(), lw=4, fmt='s', label='Det{} nsinit=1024, ns=1024'.format(k))
legend()
xlabel('Sub-band')
ylabel('TOD')
axvline(x=nus_edge[0], color='k', ls=':')
axvline(x=nus_edge[-1], color='k', ls=':')

In [ ]:
allnsubtod = [10, 5, 2]
detnum = 0

for nsubtod in allnsubtod:
    H, mm, nus_edge, nus, deltas, Delta, ptg = get_H(nsubtod, nside=256, nside_init=1024)
    mm[:,:,1:] = 0

    mytod = np.zeros((nsubtod, 3000))
    for i,h in enumerate(H.operands):
        mytod[i,:] = h(mm[i,:,:])[0,:]
    
    del H, h
    gc.collect()

    p=myplt(nus, mytod[:,detnum],deltas/2, label='Nsub={}'.format(nsubtod))
    axvspan(nus[0], nus[-1], alpha=0.1, color=p[0].get_color(), label='Actual region for Nsub={}'.format(nsubtod))

legend()
xlabel('Sub-band')
ylabel('TOD')
axvline(x=nus_edge[0], color='k', ls=':', label='Physical Band low edge')
axvline(x=nus_edge[-1], color='k', ls=':', label='Physical Band high edge')

